In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

import cv2
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random

import json

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# GPU selection (memory)
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# Data info.

width = 640
height = 480
channel = 3


# resize
r_w = 512
r_h = 512

total_pix = r_w * r_h * channel

In [4]:
data_txt = open('./data_split/data_dir_dict_0614.txt', 'r')
data_json = data_txt.read()
tr_data_dir = json.loads(data_json)

In [5]:
ev_data = np.zeros((len(tr_data_dir['test']), total_pix), dtype=np.float32)
ev_label = np.zeros((len(tr_data_dir['test']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['test'])):
    img = cv2.imread(tr_data_dir['test'][i][0])
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    ev_data[i,:] = img2.flatten()
    ev_label[i] = tr_data_dir['test'][i][1]

In [6]:
f_dir1 = '/models/ICROS/Train_9/model.ckpt-2280.meta'
f_dir2 = '/models/ICROS/Train_9/'

saver = tf.train.import_meta_graph(f_dir1)
batch = 32
ev_steps = int(ev_label.shape[0]/batch)
ev_acc = np.zeros(ev_label.shape[0])

with tf.Session() as sess:
    
    new_saver = tf.train.import_meta_graph(f_dir1)
    new_saver.restore(sess, tf.train.latest_checkpoint(f_dir2))
    graph = tf.get_default_graph()
    #for op in tf.get_default_graph().get_operations():
        #print(op.name)
        #print(op.values())
        
    data_in = graph.get_tensor_by_name("fifo_queue_DequeueUpTo:1")

    #feed_dict={data_in:ev_data[0:10], label:ev_label[0:10]}
    
    for i in range(ev_steps):
        if i < (ev_steps-1):
            feed_dict={data_in:ev_data[batch*i:batch*(i+1)]}
            label = ev_label[batch*i:batch*(i+1)].reshape(batch)
        else:
            feed_dict={data_in:ev_data[batch*i:]}
            temp = ev_label[batch*i:]
            label = temp.reshape(len(temp))
        
        prob = graph.get_tensor_by_name("softmax_tensor:0")
        classes = graph.get_tensor_by_name("ArgMax:0")
        
        result_pr = sess.run(prob, feed_dict=feed_dict)
        result_cl = sess.run(classes, feed_dict=feed_dict)
        
        print(result_pr)
        print(result_cl)
        print(label)

        acc = np.equal(result_cl, label).reshape((1,len(label)))*1
        
        print(acc)
        print(np.mean(acc))
        
        if i < (ev_steps-1):
            ev_acc[batch*i:batch*(i+1)] = acc
        else:
            ev_acc[batch*i:] = acc


INFO:tensorflow:Restoring parameters from /models/ICROS/Train_9/model.ckpt-2280
[[4.6823834e-09 1.0000000e+00]
 [1.3969163e-03 9.9860305e-01]
 [1.0000000e+00 4.9452189e-11]
 [1.0000000e+00 4.4960259e-13]
 [1.0000000e+00 2.3664044e-12]
 [7.0816117e-05 9.9992919e-01]
 [7.1474012e-08 9.9999988e-01]
 [9.6198701e-06 9.9999034e-01]
 [1.0000000e+00 7.9645335e-12]
 [2.9498443e-04 9.9970502e-01]
 [2.2864148e-03 9.9771357e-01]
 [9.9999201e-01 7.9810097e-06]
 [1.6478049e-04 9.9983513e-01]
 [1.2634668e-04 9.9987364e-01]
 [6.9716314e-14 1.0000000e+00]
 [1.0000000e+00 4.7205521e-11]
 [2.3077546e-04 9.9976927e-01]
 [7.1707092e-07 9.9999928e-01]
 [7.0451811e-02 9.2954814e-01]
 [1.2142582e-05 9.9998784e-01]
 [9.9982661e-01 1.7341842e-04]
 [1.4179513e-05 9.9998581e-01]
 [9.9965382e-01 3.4622356e-04]
 [9.9998331e-01 1.6739978e-05]
 [2.5643457e-02 9.7435659e-01]
 [9.9996173e-01 3.8308597e-05]
 [2.7355681e-13 1.0000000e+00]
 [3.7665457e-05 9.9996233e-01]
 [9.9995506e-01 4.4960569e-05]
 [5.6101341e-04 9.994

[[1.37382634e-02 9.86261785e-01]
 [1.00000000e+00 4.68339430e-12]
 [9.99540567e-01 4.59478906e-04]
 [1.28267189e-07 9.99999881e-01]
 [9.99951005e-01 4.89416707e-05]
 [1.00000000e+00 2.80361512e-09]
 [1.25666591e-03 9.98743355e-01]
 [1.00000000e+00 2.66636175e-11]
 [2.15592308e-07 9.99999762e-01]
 [8.17546534e-05 9.99918222e-01]
 [9.99990225e-01 9.79534889e-06]
 [9.99999762e-01 1.84204097e-07]
 [1.25721740e-02 9.87427831e-01]
 [9.99804437e-01 1.95623783e-04]
 [2.45928115e-13 1.00000000e+00]
 [9.78958786e-01 2.10412070e-02]
 [9.99994397e-01 5.60246917e-06]
 [1.36445290e-21 1.00000000e+00]
 [9.99997020e-01 3.00521219e-06]
 [9.99998808e-01 1.21122321e-06]
 [1.62307371e-03 9.98376966e-01]
 [9.99352753e-01 6.47205161e-04]
 [1.00000000e+00 9.44367606e-09]
 [2.56711355e-04 9.99743283e-01]
 [1.00000000e+00 1.69090436e-10]
 [7.34768389e-03 9.92652297e-01]
 [9.99999762e-01 2.89747874e-07]
 [1.00000000e+00 7.79236606e-11]
 [5.46555784e-05 9.99945283e-01]
 [9.99999881e-01 8.21759087e-08]
 [9.999890

[[9.99909997e-01 9.00430969e-05]
 [7.30769971e-05 9.99926925e-01]
 [9.99999762e-01 2.18508816e-07]
 [2.00977156e-04 9.99799073e-01]
 [1.00000000e+00 1.86264944e-11]
 [1.00000000e+00 1.45039040e-17]
 [3.17664160e-07 9.99999642e-01]
 [1.91389874e-04 9.99808609e-01]
 [9.94278789e-01 5.72113926e-03]
 [9.99998927e-01 1.05431218e-06]
 [1.15748413e-03 9.98842537e-01]
 [9.99556720e-01 4.43312689e-04]
 [1.11379904e-07 9.99999881e-01]
 [9.93610561e-01 6.38941349e-03]
 [3.38842394e-03 9.96611536e-01]
 [1.00000000e+00 3.40256925e-08]
 [5.16014989e-05 9.99948382e-01]
 [9.74059582e-01 2.59404108e-02]
 [1.00000000e+00 4.59292977e-13]
 [4.60329511e-06 9.99995351e-01]
 [1.30601275e-05 9.99986887e-01]
 [1.00000000e+00 5.01414754e-21]
 [3.78452502e-02 9.62154806e-01]
 [1.00000000e+00 3.64631170e-08]
 [9.99999404e-01 6.14520218e-07]
 [5.75546772e-12 1.00000000e+00]
 [9.53281869e-06 9.99990463e-01]
 [5.00920927e-04 9.99499083e-01]
 [1.00000000e+00 5.39386036e-10]
 [9.99904871e-01 9.51558686e-05]
 [1.757510

[[1.37316791e-04 9.99862671e-01]
 [9.99935746e-01 6.42118612e-05]
 [4.97706460e-05 9.99950171e-01]
 [9.99531269e-01 4.68799990e-04]
 [1.00000000e+00 7.06036341e-11]
 [1.22704380e-03 9.98772919e-01]
 [9.99986172e-01 1.38833975e-05]
 [7.26033400e-11 1.00000000e+00]
 [1.00000000e+00 4.18575480e-10]
 [9.99875307e-01 1.24692364e-04]
 [9.99857068e-01 1.42925710e-04]
 [1.00000000e+00 5.53192110e-11]
 [1.00000000e+00 2.56432520e-09]
 [9.99999046e-01 1.00077250e-06]
 [9.99990702e-01 9.29028829e-06]
 [4.13992995e-04 9.99586046e-01]
 [7.97114428e-03 9.92028892e-01]
 [6.23511465e-10 1.00000000e+00]
 [1.33930589e-03 9.98660684e-01]
 [1.00000000e+00 3.08661551e-30]
 [9.99743998e-01 2.56007421e-04]
 [9.99999285e-01 7.27209567e-07]
 [1.94718610e-04 9.99805272e-01]
 [4.68743779e-02 9.53125596e-01]
 [1.00000000e+00 3.03538386e-08]
 [9.99995589e-01 4.38133839e-06]
 [5.50674770e-07 9.99999404e-01]
 [9.99999762e-01 1.82151140e-07]
 [9.99928117e-01 7.18898082e-05]
 [9.99992847e-01 7.15479837e-06]
 [1.000000

In [7]:
print(len(ev_acc))
print(len(ev_label))

914
914


In [8]:
print(ev_acc[-1])
print(np.mean(ev_acc))

1.0
0.9814004376367614


In [11]:
final = np.array([0.9387, 0.9497, 0.9628, 0.9671, 0.9803, 0.9628, 0.9551, 0.8457, 0.9726, 0.9814])
print(np.mean(final))
print(np.std(final))

0.9516200000000001
0.0374712369691741
